<a href="https://colab.research.google.com/github/mayurijoshi106/mayuri_INFO5731_spring2020/blob/master/INFO5731_Assignment_Three_MJ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment Three**

In this assignment, you are required to conduct information extraction, semantic analysis based on **the dataset you collected from assignment two**. You may use scipy and numpy package in this assignment.

# **Question 1: Understand N-gram**

(45 points). Write a python program to conduct N-gram analysis based on the dataset in your assignment two:

(1) Count the frequency of all the N-grams (N=3).

(2) Calculate the probabilities for all the bigrams in the dataset by using the fomular count(w2 w1) / count(w2). For example, count(really like) / count(really) = 1 / 3 = 0.33.

(3) Extract all the **noun phrases** and calculate the relative probabilities of each review in terms of other reviews (abstracts, or tweets) by using the fomular frequency (noun phrase) / max frequency (noun phrase) on the whole dataset. Print out the result in a table with column name the all the noun phrases and row name as all the 100 reviews (abstracts, or tweets). 


In [3]:
import nltk
from nltk.util import ngrams
nltk.download('punkt')
with open("cleaneddata_assignment3.txt", "r", encoding='utf-8') as file:
  data = file.read()




[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [37]:
import pandas as pd

#Count the frequency of all the N-grams (N=3)
def extract_ngrams(data, num):
    n_grams = ngrams(nltk.word_tokenize(data), num)
    return [ ' '.join(grams) for grams in n_grams]

dict = extract_ngrams(data, 3)
df = pd.DataFrame({'tri-grams':dict})
print(df.head(11))  

               tri-grams
0       give sister cook
1      sister cook clean
2        cook clean take
3       clean take trash
4        take trash love
5    trash love facebook
6      love facebook new
7    facebook new laptop
8      new laptop bought
9   laptop bought laptop
10    bought laptop spec


In [39]:
# Calculate the probabilities for all the bigrams in the dataset by using the fomula count(w2 w1) / count(w2). For example, count(really like) / count(really) = 1 / 3 = 0.33.

dict1 = extract_ngrams(data, 2)
df = pd.DataFrame({'bi-grams':dict1})
print(df.head(11))  

# calculate the probabilities of bigrams
bigr_prob={}
for key, val in dict1.items():
  bigr_prob[key]=val/data.words.count(key[0])

         bi-grams
0     give sister
1     sister cook
2      cook clean
3      clean take
4      take trash
5      trash love
6   love facebook
7    facebook new
8      new laptop
9   laptop bought
10  bought laptop


AttributeError: ignored

AttributeError: ignored

# **Question 2: Undersand TF-IDF and Document representation**

(40 points). Starting from the documents (all the reviews, or abstracts, or tweets) collected for assignment two, write a python program: 

(1) To build the **documents-terms weights (tf*idf) matrix bold text**.

(2) To rank the documents with respect to query (design a query by yourself, for example, "An Outstanding movie with a haunting performance and best character development") by using **cosine similarity**.

In [0]:
# Write your code here





# **Question 3: Create your own training and evaluation data for sentiment analysis**

(15 points). **You dodn't need to write program for this question!** Read each review (abstract or tweet) you collected in detail, and annotate each review with a sentiment (positive, negative, or neutral). Save the annotated dataset into a csv file with three columns (first column: document_id, clean_text, sentiment), upload the csv file to GitHub and submit the file link blew. This datset will be used for assignment four: sentiment analysis and text classification. 


In [0]:
# The GitHub link of your final csv file

# Link: 